In [ ]:
import requests
import json
import os
import cmlapi

USER_QUESTION = "What is CML?"  # Update this with your own!
TEMPERATURE = 0.1
TOKEN_COUNT = 250

client = cmlapi.default_client(url=os.getenv("CDSW_API_URL").replace("/api/v1", ""), cml_api_key=os.getenv("CDSW_APIV2_KEY"))
projects = client.list_projects(include_public_projects=True, search_filter=json.dumps({"name": "Intelligent QA Chatbot with   NiFi, Pinecone, and Llama2"}))
project = projects.projects[0]

## Here we assume that only one model has been deployed in the project, if this is not true this should be adjusted (this is reflected by the placeholder 0 in the array)
model = client.list_models(project_id=project.id)
selected_model = model.models[0]

## Save the access key for the model to the environment variable of this project
MODEL_ACCESS_KEY = selected_model.access_key

MODEL_ENDPOINT = os.getenv("CDSW_API_URL").replace("https://", "https://modelservice.").replace("/api/v1", "/model?accessKey=")
MODEL_ENDPOINT = MODEL_ENDPOINT + MODEL_ACCESS_KEY

llama_sys = f"<s>[INST]You are a helpful, respectful and honest assistant. If you are unsure about an answer, truthfully say \"I don't know\"."    
question_and_context = f"{llama_sys} \n[INST] {USER_QUESTION} [/INST]"
    
try:
    # Build a request payload for CML hosted model
    data={ "request": {"prompt":question_and_context,"temperature":TEMPERATURE,"max_new_tokens":TOKEN_COUNT,"repetition_penalty":1.0} }
    
    r = requests.post(MODEL_ENDPOINT, data=json.dumps(data), headers={'Content-Type': 'application/json'})
    
    # Logging
    print(f"Request: {data}")
    print(f"Response: {r.json()}")
    
    no_inst_response = str(r.json()['response']['prediction']['response'])[len(question_and_context)-6:]
        
    return no_inst_response
    
except Exception as e:
    print(e)
    return e

# Check if the request was successful
if response.status_code == 200:
    print(f"Success! Received response: {response.json()}")
else:
    print(f"Failed to make request. Status code: {response.status_code}, Response: {response.text}")


https://127.0.0.1:8090/
